# 读取数据

In [2]:
import pandas as pd
df = pd.read_csv('**.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Columns: 9036 entries, core_ADC_original_shape_Elongation to Y
dtypes: float64(8839), int64(196), object(1)
memory usage: 13.0+ MB


# 处理缺失值

In [3]:
df_drop = df.dropna(axis=0)
df_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166 entries, 0 to 185
Columns: 9036 entries, core_ADC_original_shape_Elongation to Y
dtypes: float64(8839), int64(196), object(1)
memory usage: 11.4+ MB


In [4]:
import numpy as np
df_drop.replace([np.inf, -np.inf],np.nan,inplace=True)
df_drop.fillna(0,inplace=True)
df_drop.info()

D:\Python3\lib\site-packages\pandas\core\frame.py:3798: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
D:\Python3\lib\site-packages\pandas\core\frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 166 entries, 0 to 185
Columns: 9036 entries, core_ADC_original_shape_Elongation to Y
dtypes: float64(8839), int64(196), object(1)
memory usage: 11.4+ MB


# 赋标签

In [5]:
class_mapping = {label: idx for idx, label in enumerate(np.unique(df_drop['Y']))}
class_mapping

{'NS': 0, 'S': 1}

# X和y选取

In [6]:
from sklearn.preprocessing import LabelEncoder
df_drop['Y'] = df_drop['Y'].map(class_mapping)
y = LabelEncoder().fit_transform(df_drop['Y'].values)
X = df_drop.iloc[:, 0:9035].values
print(y.shape)
print(X.shape)

(166,)
(166, 9035)


D:\Python3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# 获取特征名称

In [7]:
feat_labels = df_drop.columns[0:9035]

# 数据标准化

In [8]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_std = stdsc.fit_transform(X)
print(X_std.shape)

(166, 9035)


# 随机森林函数的参数展示

In [9]:
from sklearn.ensemble import RandomForestClassifier
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

# 选出最佳的树的数量和叶子数量，，有点久，一般树选500，叶子最少选10就行

In [10]:
df_drop.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
sample_leaf_options = list(range(1, 500, 3))
n_estimators_options = list(range(1, 1000, 5))
groud_truth = df_drop['Y'][141:]
results = []
for leaf_size in sample_leaf_options:
    for n_estimators_size in n_estimators_options:
        alg = RandomForestClassifier(max_features='sqrt',min_samples_leaf=leaf_size,n_estimators=n_estimators_size,random_state=50)
        alg.fit(df_drop.drop("Y", axis=1)[:140], df_drop['Y'][:140])
        predict = alg.predict(df_drop.drop("Y", axis=1)[141:])
        results.append((leaf_size, n_estimators_size, (groud_truth == predict).mean()))
        print((groud_truth == predict).mean())
print(max(results, key=lambda x: x[2]))

0.64
0.56
0.48
0.52
0.44
0.4
0.4
0.44
0.44
0.44
0.44
0.44
0.44
0.44
0.48
0.48
0.48
0.48
0.48
0.52
0.52
0.48
0.48
0.44
0.48
0.44
0.44
0.44
0.44
0.48
0.48
0.48
0.48
0.48
0.52
0.48
0.52
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.44
0.44
0.44
0.48
0.48
0.48
0.44
0.48
0.44
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.44
0.48
0.44
0.48
0.48
0.48
0.48
0.48
0.44
0.44
0.44
0.44
0.44
0.44
0.44
0.44
0.44
0.48
0.48
0.48
0.44
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.48
0.

# 随机森林算法评估特征排名

In [9]:
forest = RandomForestClassifier(bootstrap=True,criterion='gini',oob_score=True,n_jobs=-1,
                                max_features='sqrt',n_estimators=500,min_samples_leaf=7,
                                random_state=50)
forest.fit(X_std, y)
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(X_std.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))

 1) core_T2-FLAIR_wavelet-LL_glcm_SumEntropy 0.002833
 2) core_T2-FLAIR_wavelet-LL_glcm_MaximumProbability 0.002500
 3) core_CBV_original_gldm_LargeDependenceLowGrayLevelEmphasis 0.002347
 4) core_CBV_original_gldm_DependenceVariance 0.002337
 5) core_T2WI_log-sigma-5-0-mm-3D_glcm_ClusterShade 0.002218
 6) core_T2-FLAIR_wavelet-LL_glcm_JointEnergy 0.002194
 7) core_T2-FLAIR_wavelet-LL_glcm_JointEntropy 0.002143
 8) core_CBV_squareroot_gldm_LargeDependenceLowGrayLevelEmphasis 0.002025
 9) core_T1-MPRAGE_wavelet-HLH_glrlm_LongRunHighGrayLevelEmphasis 0.001889
10) core_T2-FLAIR_wavelet-LL_glszm_ZoneVariance 0.001862
11) core_T2-FLAIR_wavelet-LL_glrlm_ShortRunEmphasis 0.001696
12) core_T2WI_wavelet-LH_glcm_Id   0.001689
13) core_CBV_wavelet-LL_firstorder_10Percentile 0.001676
14) core_T2-FLAIR_wavelet-LL_firstorder_Entropy 0.001645
15) core_CBV_original_glcm_JointEnergy 0.001607
16) core_T2-FLAIR_original_glcm_InverseVariance 0.001574
17) core_CBV_original_glrlm_LongRunLowGrayLevelEmphasis